In [34]:
import pandas as pd
import numpy as np
df1 = pd.read_csv('data_motores.csv')
pd.set_option('display.max_columns', None)

In [35]:
df1["Eficiencia (%)"]               = df1["Eficiencia (%)"]/100
df1["Factor de carga maximo"]       = df1["Factor de carga maximo"]/100
df1['Potencia Entrada (W)']         = (df1['Potencia (HP)'] * 746 * (df1['Factor de carga maximo'])) / (df1['Eficiencia (%)'])
df1['Corriente a 220V (A)']         = df1['Potencia Entrada (W)'] / (np.sqrt(3) * 220 * df1['Factor de potencia'])
df1['Corriente a 380V (A)']         = df1['Potencia Entrada (W)'] / (np.sqrt(3) * 380 * df1['Factor de potencia'])
df1['Corriente a 220V (A) +25%']    = df1['Corriente a 220V (A)']*1.25
df1['Corriente a 380V (A) +25%']    = df1['Corriente a 380V (A)']*1.25
df1['Velocidad síncrona (RPM)']     = 3600

In [36]:
catalogo                            = pd.read_csv('catalogo_motores.csv')
catalogo['Potencia HP']             = catalogo['Potencia HP'].astype(int)
df2                                 = pd.merge(df1, catalogo[['Potencia HP', 'Velocidad rpm']], left_on='Potencia (HP)', right_on='Potencia HP', how='left')
df2.drop(columns=['Potencia HP'], inplace=True)
df2                                 = df2.rename(columns={'Velocidad rpm': 'Velocidad real (RPM)'})
df2['Deslizamiento (RPM)']          = df2['Velocidad síncrona (RPM)'] - df2['Velocidad real (RPM)']
df2['Deslizamiento (%)']            = ((df2['Velocidad síncrona (RPM)'] - df2['Velocidad real (RPM)']) / df2['Velocidad síncrona (RPM)'])*100
df2['Torque (N.m)']                 = (df2['Potencia (HP)'] * 746 * (df1['Factor de carga maximo'])) / ((df2['Velocidad real (RPM)'] * 2 * np.pi)/60)
df2['theta (°)']                    = np.degrees(np.arccos(df2['Factor de potencia']))
df2['Real Corriente 220V']          = df2['Corriente a 220V (A) +25%'] * np.cos(np.radians(df2['theta (°)']))
df2['Imaginaria Corriente 220V']    = df2['Corriente a 220V (A) +25%'] * np.sin(np.radians(df2['theta (°)']))
df2['Real Corriente 380V']          = df2['Corriente a 380V (A) +25%'] * np.cos(np.radians(df2['theta (°)']))
df2['Imaginaria Corriente 380V']    = df2['Corriente a 380V (A) +25%'] * np.sin(np.radians(df2['theta (°)']))
df2["kw/año"]                       = (df2["Potencia Entrada (W)"]/1000) * 6696
df2["costo/año"]                    = df2["kw/año"] * 1009.04

df2["cambio FC"]                    = np.where(df2["Factor de carga maximo"] < 0.8, "Si", "No")
df2["cambio Eficiencia"]            = np.where(df2["Eficiencia (%)"] < 0.85, "Si", "No")
df2["cambio"]                       = np.where((df2["cambio FC"] == "Si") | (df2["cambio Eficiencia"] == "Si"), "Si", "No")

In [37]:
df2.to_csv('data_motores_procesada.csv', index=False)

# PARA 220V

Magnitud de corriente a 220V para determinar el total de la corriente que se usará para dimensionar el grosor del cable sobre la Planta 1

In [38]:
Corriente_Planta_1_220V = np.sqrt((df2[df2['Planta'] == 'Planta 1']['Real Corriente 220V'].sum())**2 + (df2[df2['Planta'] == 'Planta 1']['Imaginaria Corriente 220V'].sum())**2)
print(f'Magnitud de corriente a 220V para los motores de la planta 1: {Corriente_Planta_1_220V} A')

Magnitud de corriente a 220V para los motores de la planta 1: 740.8613845652662 A


Magnitud de corriente a 220V para determinar el total de la corriente que se usará para dimensionar el grosor del cable sobre la Planta 2

In [39]:
Corriente_Planta_2_220V = np.sqrt((df2[df2['Planta'] == 'Planta 2']['Real Corriente 220V'].sum())**2 + (df2[df2['Planta'] == 'Planta 2']['Imaginaria Corriente 220V'].sum())**2)
print(f'Magnitud de corriente a 220V para los motores de la planta 2: {Corriente_Planta_2_220V} A')

Magnitud de corriente a 220V para los motores de la planta 2: 1055.846103440497 A


Magnitud de corriente a 220V para determinar el total de la corriente que se usará para dimensionar el grosor del cable sobre la Planta 3

In [40]:
Corriente_Planta_3_220V = np.sqrt((df2[df2['Planta'] == 'Planta 3']['Real Corriente 220V'].sum())**2 + (df2[df2['Planta'] == 'Planta 3']['Imaginaria Corriente 220V'].sum())**2)
print(f'Magnitud de corriente a 220V para los motores de la planta 3: {Corriente_Planta_3_220V} A')

Magnitud de corriente a 220V para los motores de la planta 3: 1184.1293915625126 A


Magnitud de corriente a 220V para determinar el total de la corriente que se usará para dimensionar el grosor del cable sobre la Planta 4

In [41]:
Corriente_Planta_4_220V = np.sqrt((df2[df2['Planta'] == 'Planta 4']['Real Corriente 220V'].sum())**2 + (df2[df2['Planta'] == 'Planta 4']['Imaginaria Corriente 220V'].sum())**2)
print(f'Magnitud de corriente a 220V para los motores de la planta 4: {Corriente_Planta_4_220V} A')

Magnitud de corriente a 220V para los motores de la planta 4: 774.7133890486778 A


Magnitud de corriente a 220V para determinar el total de la corriente que se usará para dimensionar el grosor del cable sobre la Planta 5

In [42]:
Corriente_Planta_5_220V = np.sqrt((df2[df2['Planta'] == 'Planta 5']['Real Corriente 220V'].sum())**2 + (df2[df2['Planta'] == 'Planta 5']['Imaginaria Corriente 220V'].sum())**2)
print(f'Magnitud de corriente a 220V para los motores de la planta 5: {Corriente_Planta_5_220V} A')

Magnitud de corriente a 220V para los motores de la planta 5: 1396.9715056603966 A


# PARA 380V

Magnitud de corriente a 380V para determinar el total de la corriente que se usará para dimensionar el grosor del cable sobre la Planta 1

In [43]:
Corriente_Planta_1_380V = np.sqrt((df2[df2['Planta'] == 'Planta 1']['Real Corriente 380V'].sum())**2 + (df2[df2['Planta'] == 'Planta 1']['Imaginaria Corriente 380V'].sum())**2)
print(f'Magnitud de corriente a 380V para los motores de la planta 1: {Corriente_Planta_1_380V} A')

Magnitud de corriente a 380V para los motores de la planta 1: 428.91974895883834 A


Magnitud de corriente a 380V para determinar el total de la corriente que se usará para dimensionar el grosor del cable sobre la Planta 2

In [44]:
Corriente_Planta_2_380V = np.sqrt((df2[df2['Planta'] == 'Planta 2']['Real Corriente 380V'].sum())**2 + (df2[df2['Planta'] == 'Planta 2']['Imaginaria Corriente 380V'].sum())**2)
print(f'Magnitud de corriente a 380V para los motores de la planta 2: {Corriente_Planta_2_380V} A')

Magnitud de corriente a 380V para los motores de la planta 2: 611.2793230444981 A


Magnitud de corriente a 380V para determinar el total de la corriente que se usará para dimensionar el grosor del cable sobre la Planta 3

In [45]:
Corriente_Planta_3_380V = np.sqrt((df2[df2['Planta'] == 'Planta 3']['Real Corriente 380V'].sum())**2 + (df2[df2['Planta'] == 'Planta 3']['Imaginaria Corriente 380V'].sum())**2)
print(f'Magnitud de corriente a 380V para los motores de la planta 3: {Corriente_Planta_3_380V} A')

Magnitud de corriente a 380V para los motores de la planta 3: 685.5485951151388 A


Magnitud de corriente a 380V para determinar el total de la corriente que se usará para dimensionar el grosor del cable sobre la Planta 4

In [46]:
Corriente_Planta_4_380V = np.sqrt((df2[df2['Planta'] == 'Planta 4']['Real Corriente 380V'].sum())**2 + (df2[df2['Planta'] == 'Planta 4']['Imaginaria Corriente 380V'].sum())**2)
print(f'Magnitud de corriente a 380V para los motores de la planta 4: {Corriente_Planta_4_380V} A')

Magnitud de corriente a 380V para los motores de la planta 4: 448.51827787028714 A


Magnitud de corriente a 380V para determinar el total de la corriente que se usará para dimensionar el grosor del cable sobre la Planta 5

In [47]:
Corriente_Planta_5_380V = np.sqrt((df2[df2['Planta'] == 'Planta 5']['Real Corriente 380V'].sum())**2 + (df2[df2['Planta'] == 'Planta 5']['Imaginaria Corriente 380V'].sum())**2)
print(f'Magnitud de corriente a 380V para los motores de la planta 5: {Corriente_Planta_5_380V} A')

Magnitud de corriente a 380V para los motores de la planta 5: 808.7729769612822 A


Debido a que existen consumos auxiliares, se tiene que estos consumen 480kW, se puede partir de que poseen un factor de potencia de FP = 1, por lo tanto es pertinente calcular su consumo de corriente a 220V que es al voltaje al cual se encuentran conectados, suponiendo que son cargas puramente resistivas, estas no cuentan con desfase por lo tanto no poseen parte imaginaria, por lo que:

In [54]:
FP_aux = 1
Corriente_Auxiliares = 480000/(np.sqrt(3)*220*FP_aux)
print(f'Magnitud de corriente a 220V para los Auxiliares: {Corriente_Auxiliares} A')

Magnitud de corriente a 220V para los Auxiliares: 1259.6733145955473 A


## CAMBIO DE MOTORES ##

In [48]:
#calculo de 11 meses, donde se trabaja continuamente 16 horas al dia de lunes a sabado
diasXmes        = 24
horasXdia       = 16
horasXmes       = diasXmes * horasXdia
meses           = 11
horasX11meses   = horasXmes * meses

#calculo para el mes de diciembre
diasDiciembre   = 15
horasDiciembre  = diasDiciembre * horasXdia

#total horas por año
TOTALHORAS     = horasX11meses + horasDiciembre

Dado un transformador de 13200/220V, según el documento de publicación de tarifas del mes de mayo de 2024, se tiene que para un transformador de este tipo se encuentra en el Nivel II de consumo por lo que la tarifa a pagar por poseer un contador antes del transformador es de:

In [49]:
TARIFA_T1 = (914.75 + 918.01)/2
TARIFA_T1

916.38

Dado un transformador de 44000/220V, según el documento de publicación de tarifas del mes de mayo de 2024, se tiene que para un transformador de este tipo se encuentra en el Nivel III de consumo por lo que la tarifa a pagar por poseer un contador antes del transformador es de:

In [50]:
TARIFA_T2 = (798.52 + 801.84)/2
TARIFA_T2

800.1800000000001

In [51]:
df_catalogo = pd.read_csv("catalogo_motores_completo.csv")
def cambio(fc,potencia_hp,velocidad_sincronica,potencia_actual):
    #factor de carga, potencia_hp, potencia_actual
    costoXaño = (potencia_actual / 1000) * TOTALHORAS * TARIFA_T1 #Reemplazar TARIFA_T1 por TARIFA_T2 si se desea calcular con la tarifa T2
    df_filtro                           = df_catalogo[(df_catalogo["Potencia HP"] == potencia_hp) & (df_catalogo["rpm"] == velocidad_sincronica)].copy()
    df_filtro['Potencia Entrada (W)']   = (df_filtro ['Potencia HP'] * 746 * (fc)) / (df_filtro ['Eficiencia %']/100)
    df_filtro["kw/año"]                 = (df_filtro["Potencia Entrada (W)"]/1000) * TOTALHORAS
    df_filtro["costo/año"]              = df_filtro["kw/año"] * TARIFA_T1
    df_filtro["costo_anterior"]         = costoXaño
    df_filtro["ahorroXmes"]             = (df_filtro["costo_anterior"] - df_filtro["costo/año"]) / 12
    df_filtro["tiempo_retribucion"]     = df_filtro["costo_Pesos"] / df_filtro["ahorroXmes"]
    return df_filtro

In [52]:
#factor de carga, potencia_hp, potencia_actual
df_cambio = cambio(0.8,50,3600,35105.882352941175)
df_cambio

,No. de Depósito,Referencia,Potencia kW,Potencia HP,Tamaño constructivo,Velocidad rpm,Torque Nm,Corriente nominal 220VDD A,Corriente nominal 380VYY A,Corriente nominal 440VD A,Eficiencia %,Peso neto (IM B3) kg,Precio de lista USD,rpm,polos,costo_Pesos,(%) deslizamiento,Potencia Entrada (W),kw/año,costo/año,costo_anterior,ahorroXmes,tiempo_retribucion
13,100664589,1LE0143-2BA16-4AA4-Z D80+D81,37.0,50.0,225M,3570,99.0,118.0,68.0,59.0,93.0,320.0,4502.5,3600,2,17487710.0,0.833333,32086.021505,143232.0,1.312549e+08,1.436083e+08,1.029451e+06,16.987422


In [53]:
df2.head()

,Planta,Distancia,Motor,Potencia (HP),Eficiencia (%),Factor de carga maximo,Factor de potencia,Potencia Entrada (W),Corriente a 220V (A),Corriente a 380V (A),Corriente a 220V (A) +25%,Corriente a 380V (A) +25%,Velocidad síncrona (RPM),Velocidad real (RPM),Deslizamiento (RPM),Deslizamiento (%),Torque (N.m),theta (°),Real Corriente 220V,Imaginaria Corriente 220V,Real Corriente 380V,Imaginaria Corriente 380V,kw/año,costo/año,cambio FC,cambio Eficiencia,cambio
0,Planta 1,200,1,30,0.70,0.8,0.74,25577.142857,90.706206,52.514119,113.382757,65.642649,3600,3540,60,1.666667,48.296781,42.268584,83.903240,76.262021,48.575560,44.151697,171264.548571,1.728128e+08,No,Si,Si
1,Planta 2,300,1,40,0.79,0.8,0.82,30217.721519,96.708479,55.989120,120.885599,69.986399,3600,3565,35,0.972222,63.944126,34.915206,99.126191,69.190507,57.388848,40.057662,202337.863291,2.041670e+08,No,Si,Si
2,Planta 3,350,1,60,0.85,0.6,0.78,31595.294118,106.302748,61.543696,132.878435,76.929620,3600,3570,30,0.833333,71.836389,38.739425,103.645179,83.152602,60.005104,48.140980,211562.089412,2.134746e+08,Si,No,Si
3,Planta 4,300,1,30,0.82,0.6,0.78,16375.609756,55.095936,31.897647,68.869921,39.872059,3600,3540,60,1.666667,36.222586,38.739425,53.718538,43.097385,31.100206,24.951118,109651.082927,1.106423e+08,Si,Si,Si
4,Planta 5,500,1,100,0.91,0.6,0.81,49186.813187,159.360381,92.261273,199.200476,115.326591,3600,3570,30,0.833333,119.727315,35.904069,161.352385,116.817110,93.414539,67.630959,329354.901099,3.323323e+08,Si,No,Si
